# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries

import pickle
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, auc, classification_report, make_scorer
import nltk
from sqlalchemy import create_engine, inspect
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
engine = create_engine('sqlite:///DesasterMessages.db')
engine.table_names()

['DesasterMessages']

In [37]:
# load data from database
engine = create_engine('sqlite:///DesasterMessages.db')
df = pd.read_sql_table(table_name='DesasterMessages', con = engine)
X = df.iloc[:,1]
Y = df.iloc[:,4:]
category_names = list(Y.columns)

In [38]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [39]:
def tokenize(text):
    tokens = word_tokenize(text.lower())
    
    Lemmatizer = WordNetLemmatizer()
    for token in tokens:
        token = Lemmatizer.lemmatize(token).strip()
        
    return tokens   

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [40]:
pipeline = Pipeline([('Features', Pipeline([
                                ('count_vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer())])),
                    ('Classifier', MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = .3, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('Features', Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [44]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_test)
#type_of_target(y_pred)

'multiclass-multioutput'

In [52]:
np.hstack(y_pred)[2]

0

In [56]:
len(np.hstack(y_test.values))

283140

In [58]:
Y.columns.values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object)

In [59]:
y_pred = pipeline.predict(X_test)

print(classification_report(np.hstack(y_test.values), np.hstack(y_pred), target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.95      0.99      0.97    258341
               request       0.80      0.44      0.57     24741
                 offer       0.83      0.09      0.16        58

           avg / total       0.94      0.94      0.93    283140



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))


In [60]:
classification_report(y_test.values, y_pred)

ValueError: Unknown label type: (array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]))

In [29]:
y_pred = pipeline.predict(X_test)

for i in np.arange(len(category_names)):
    print('Category: ' + category_names[i])
    print(classification_report(np.array(y_test.iloc[:,i]), y_pred[:,i]))

Category: related
             precision    recall  f1-score   support

          0       0.60      0.33      0.42      1873
          1       0.81      0.93      0.86      5934
          2       0.80      0.07      0.13        58

avg / total       0.76      0.78      0.75      7865

Category: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      6533
          1       0.83      0.37      0.51      1332

avg / total       0.87      0.88      0.86      7865

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

Category: aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      4646
          1       0.74      0.49      0.59      3219

avg / total       0.73      0.72      0.71      7865

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
def scoring_metric(y_true, y_pred):
    f1_list = []
    for i in np.arange(y_pred.shape[1]):
        f1_list.append(f1_score(np.array(y_true.values)[:,i], y_pred[:,i], average = 'micro'))
    
    score = np.mean(f1_list)
    
    return score      

In [10]:
parameters = {
    'Features__count_vect__max_df': (.5,1),
    'Features__count_vect__max_features': (None, 1000),
    'Features__tfidf__use_idf': (True, False)
}

scorer = make_scorer(scoring_metric)
cv = GridSearchCV(pipeline, param_grid=parameters, scoring = scorer)

cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('Features', Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'Features__count_vect__max_df': (0.5, 1), 'Features__count_vect__max_features': (None, 1000), 'Features__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(scoring_metric), verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
category_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [ ]:
for i in np.arange(len(category_names)):
    print('Category: ' + category_names[i])
    print(classification_report(np.array(y_test.iloc[:,i]), y_pred[:,i]))

In [16]:
#y_pred_test = cv.predict(X_test)

# classification report on test data
#print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

#print(classification_report(y_test.values, y_pred_test, target_names = category_names))

ValueError: Unknown label type: (array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [34]:
# save model in pickle file
with open('classifier.pkl', 'wb') as f:
    pickle.dump(pipeline, f)



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.